In [85]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/kostin_001/anaconda3/envs/thesis/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [86]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [87]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [88]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(40, 60, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3) ))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [89]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [90]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [91]:
batch_size = 16

In [92]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        #rotation_range=30
        #width_shift_range=0.1,
        #height_shift_range=0.1
)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [93]:
train_generator = train_datagen.flow_from_directory(
        'data5/train',  # this is the target directory
        target_size=(40, 60),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale'
)  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data5/val',
        target_size=(40, 60),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale'
)

Found 19919 images belonging to 2 classes.
Found 105000 images belonging to 2 classes.


In [94]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=19919 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=105000 // batch_size)
model.save_weights('first_try.h5')

Epoch 1/5
1244/1244 [==============================] - 78s 63ms/step - loss: 0.0892 - acc: 0.9696 - val_loss: 0.2039 - val_acc: 0.9591
Epoch 2/5
1244/1244 [==============================] - 77s 62ms/step - loss: 0.0511 - acc: 0.9867 - val_loss: 0.2082 - val_acc: 0.9663
Epoch 3/5
1244/1244 [==============================] - 77s 62ms/step - loss: 0.0390 - acc: 0.9909 - val_loss: 0.2547 - val_acc: 0.9624
Epoch 4/5
1244/1244 [==============================] - 77s 62ms/step - loss: 0.0375 - acc: 0.9917 - val_loss: 1.5181 - val_acc: 0.8673
Epoch 5/5
1244/1244 [==============================] - 77s 62ms/step - loss: 0.0414 - acc: 0.9921 - val_loss: 0.3178 - val_acc: 0.9607


In [95]:
with open('model.json', 'w') as f:
    f.write(model.to_json())